## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,84.16,82,32,11.01,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.33,98,20,3.44,few clouds
3,3,San Quintin,MX,30.4833,-115.9500,66.20,64,5,13.80,clear sky
4,4,Pacific Grove,US,36.6177,-121.9166,60.91,81,100,8.01,overcast clouds


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?95


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,84.16,82,32,11.01,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.33,98,20,3.44,few clouds
5,5,Alofi,NU,-19.0595,-169.9187,71.49,100,95,2.30,overcast clouds
10,10,Kirakira,SB,-10.4544,161.9205,76.89,84,98,1.16,overcast clouds


In [52]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [53]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,84.16,82,32,11.01,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.33,98,20,3.44,few clouds
5,5,Alofi,NU,-19.0595,-169.9187,71.49,100,95,2.30,overcast clouds
10,10,Kirakira,SB,-10.4544,161.9205,76.89,84,98,1.16,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
680,680,Kyaikto,MM,17.3000,97.0167,78.01,91,99,6.98,overcast clouds
682,682,Ranong,TH,9.9658,98.6348,79.65,85,100,13.09,overcast clouds
684,684,Barcelos,PT,41.5388,-8.6151,77.14,48,7,5.88,clear sky
687,687,Kattivakkam,IN,13.2167,80.3167,87.89,83,48,15.23,scattered clouds


In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
1,Puerto Ayora,EC,84.16,scattered clouds,-0.7393,-90.3518,
2,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
5,Alofi,NU,71.49,overcast clouds,-19.0595,-169.9187,
10,Kirakira,SB,76.89,overcast clouds,-10.4544,161.9205,
14,Bengkulu,ID,75.88,overcast clouds,-3.8004,102.2655,
15,Xuddur,SO,79.25,broken clouds,4.1213,43.8894,
17,Ibra,OM,84.72,clear sky,22.6906,58.5334,
21,Bandarbeyla,SO,80.33,broken clouds,9.4942,50.8122,
23,Salalah,OM,84.29,broken clouds,17.0151,54.0924,


In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:  
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [56]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Puerto Ayora,EC,84.16,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
5,Alofi,NU,71.49,overcast clouds,-19.0595,-169.9187,Taloa Heights
10,Kirakira,SB,76.89,overcast clouds,-10.4544,161.9205,SanBiz Lodge


In [57]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd> F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))